The following code takes the saved LLM response files, matches the stimuli file to the responses, adds IDs by each LLM conversation, adds context and type, connects the tables and reorders them based on the "Selected word" and "Context"

In [ ]:
import pandas as pd

In [ ]:
class StimuliMatcher:
    def __init__(self, stimuli_path, interpretation_paths, output_path):
        self.stimuli_path = stimuli_path
        self.interpretation_paths = interpretation_paths
        self.output_path = output_path

        self.stimuli_df = pd.read_excel(self.stimuli_path)

    def normalize(self, sentence):
        """Cleans sentence for reliable matching."""
        return str(sentence).strip().replace('"', '').replace('[', '').replace(']', '')

    def match_and_merge(self, response_df):
        """Matches a single interpretation file with the stimuli."""
        merged_rows = []

        for _, stim_row in self.stimuli_df.iterrows():
            stim_sentence = self.normalize(stim_row['sentence'])

            for _, resp_row in response_df.iterrows():
                resp_sentence = self.normalize(resp_row['Sentence'])

                if stim_sentence == resp_sentence:
                    merged_row = resp_row.copy()
                    merged_row['Context'] = stim_row['Meaning/context']
                    merged_row['Type'] = stim_row['type']  # Note: 'type' becomes 'Type'
                    merged_rows.append(merged_row)
                    break  # Stop looking once match is found

        return merged_rows

    def create_augmented_file(self):

        all_merged = []

        for file_index, path in enumerate(self.interpretation_paths):
            """Processes all interpretation files and saves combined output."""
            response_df = pd.read_excel(path)
            # Calculate base LLM ID for this file (0-1, 1-13, 2-25, etc.)
            base_llm_id = file_index * 12 + 1
            # Add LLM_ID column based on row position (every 10 rows get same ID)
            response_df['LLM_ID'] = [f'LLM{base_llm_id + (i // 10)}' for i in range(len(response_df))]
            matched_rows = self.match_and_merge(response_df)
            all_merged.extend(matched_rows)

        combined_df = pd.DataFrame(all_merged)

        # Reorder by 'Selected Word' and 'Meaning/context' for easier handling
        if 'Selected Word' in combined_df.columns:
            combined_df.sort_values(by=['Selected Word', 'Context'], inplace=True)

        combined_df.to_excel(self.output_path, index=False)
        print(f"Saved combined and reordered file to '{self.output_path}'")

interpretation_paths = [f"interpretation_results_sentences_{i}.xlsx" for i in range(1, 6)]

matcher = StimuliMatcher(
    stimuli_path="cleared_stimuli.xlsx",
    interpretation_paths = interpretation_paths,
    output_path="combined_llm_responses.xlsx"
)

matcher.create_augmented_file()

Saved combined and reordered file to 'combined_llm_responses.xlsx'


Clearing human responses. Since the human resposne file had a different structure, first it is necessary to create question codes for the question retrieval of each interpretation

In [ ]:
compound_words = ['jailbird', 'blackmail', 'mushroom', 'claptrap', 'humbug', 'pigeonhole', 'township', 'slapstick', 'chopstick', 'touchstone', 'meantime', 'carpet', 'message', 'chinaware', 'warsaw', 'psychopath', 'airline', 'highlight', 'sidekick', 'rugrat' ]
pseudo_words = ['coaster', 'river', 'litter', 'bower', 'flower', 'brother', 'horny', 'busy', 'potage', 'footage', 'petal', 'marshal', 'tactic', 'delight', 'unionizing', 'better', 'undies', 'deliver', 'early', 'summer']
all_words = compound_words + pseudo_words

code_to_info= {}

for i, word in enumerate(all_words):
    word_index = (i % 20) + 1  # 1 to 20
    prefix = 'c' if i < 20 else 'p'
    for context in range(1, 4):
        code = f"{prefix}-{word_index}-{context}"
        code_to_info[code] = {'word': word, 'context': context}
print(code_to_info)

{'c-1-1': {'word': 'jailbird', 'context': 1}, 'c-1-2': {'word': 'jailbird', 'context': 2}, 'c-1-3': {'word': 'jailbird', 'context': 3}, 'c-2-1': {'word': 'blackmail', 'context': 1}, 'c-2-2': {'word': 'blackmail', 'context': 2}, 'c-2-3': {'word': 'blackmail', 'context': 3}, 'c-3-1': {'word': 'mushroom', 'context': 1}, 'c-3-2': {'word': 'mushroom', 'context': 2}, 'c-3-3': {'word': 'mushroom', 'context': 3}, 'c-4-1': {'word': 'claptrap', 'context': 1}, 'c-4-2': {'word': 'claptrap', 'context': 2}, 'c-4-3': {'word': 'claptrap', 'context': 3}, 'c-5-1': {'word': 'humbug', 'context': 1}, 'c-5-2': {'word': 'humbug', 'context': 2}, 'c-5-3': {'word': 'humbug', 'context': 3}, 'c-6-1': {'word': 'pigeonhole', 'context': 1}, 'c-6-2': {'word': 'pigeonhole', 'context': 2}, 'c-6-3': {'word': 'pigeonhole', 'context': 3}, 'c-7-1': {'word': 'township', 'context': 1}, 'c-7-2': {'word': 'township', 'context': 2}, 'c-7-3': {'word': 'township', 'context': 3}, 'c-8-1': {'word': 'slapstick', 'context': 1}, 'c-8-

In [ ]:
import pandas as pd
#!pip install openpyxl

class HumanResponseProcessor:
    def __init__(self, response_file, code_to_info, compound_words, pseudo_words):
        self.response_file = response_file
        self.code_to_info = code_to_info
        self.compound_words = set(compound_words)
        self.pseudo_words = set(pseudo_words)

    def transform_responses(self):
        df = pd.read_excel(self.response_file)

        # Drop the second row (usually question text, not actual data)
        df = df.drop(index=0).reset_index(drop=True)

        results = []
        for code, info in self.code_to_info.items():
            if code not in df.columns:
                continue

            confidence_col = code + '-c_1'

            for row_idx, row in df.iterrows():
                interpretation = row.get(code)
                confidence = row.get(confidence_col, None)

                if pd.notna(interpretation):
                    results.append({
                        "Participant ID": row_idx + 1, # Add a row index as a participants ID since all the data is saved as a row for each participant
                        "Selected word": info['word'],
                        "Interpretation": interpretation,
                        "Confidence": confidence,
                        "Context": info['context']
                    })

        return pd.DataFrame(results)

    def clean_and_label(self, df):
        # Add "Type" based on word
        def get_word_type(word):
            if word in self.compound_words:
                return "compound"
            elif word in self.pseudo_words:
                return "pseudo-derived"
            else:
                return "unknown"

        df["Type"] = df["Selected word"].apply(get_word_type)

        # Clean "Confidence"
        def clean_confidence(val):
            if pd.isna(val):
                return None
            val_str = str(val).strip()
            return int(val_str[0]) if val_str and val_str[0].isdigit() else None

        df["Confidence"] = df["Confidence"].apply(clean_confidence)

        return df

    def process_and_save(self, output_path):
        df_transformed = self.transform_responses()
        df_cleaned = self.clean_and_label(df_transformed)
        df_cleaned.to_excel(output_path, index=False)
        print(f"Final cleaned responses saved to: {output_path}")

# Process the file
processor = HumanResponseProcessor(
    response_file="Novel+word+survey_April+11,+2025_19.54.xlsx",
    code_to_info=code_to_info,
    compound_words=compound_words,
    pseudo_words=pseudo_words
)

processor.process_and_save("final_human_responses.xlsx")

Final cleaned responses saved to: final_human_responses.xlsx


The following code is for getting similarity scores for the cleared data

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import itertools

Similarity scores of Target/golden interpretation vs. LLM/Human interpretations

In [ ]:
# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Choose the correct file
human_response_path = "final_human_responses.xlsx"
llm_response_path = "combined_llm_responses_with_IDs_cleared.xlsx"#"final_llm_responses.xlsx"
human_output_path = "final_human_responses_with_scores.xlsx"
llm_output_path = "final_llm_responses_with_scores.xlsx"

chosen_response_path = llm_response_path #Change the desired path here
chosen_output_path = llm_output_path

# Load files
stimuli_df = pd.read_excel("cleared_stimuli.xlsx")
df = pd.read_excel(chosen_response_path) #Input the path given the task

# Initialize a new column
df["InterpretationScore"] = None

# Go through each golden interpretation
for _, stim_row in stimuli_df.iterrows():
    if stim_row["Meaning/context"] != "New":
        continue

    golden_sentence = stim_row["sentence"]
    target_word = stim_row["word"]

    # Embed golden sentence once
    golden_emb = model.encode(golden_sentence, convert_to_tensor=True)

    # Find matching LLM responses
    mask = df["Selected word"] == target_word
    for idx, row in df[mask].iterrows():
        interpretation = row["Interpretation"]

        # Handle empty interpretations
        if pd.isna(interpretation) or not interpretation.strip():
            continue

        interp_emb = model.encode(interpretation, convert_to_tensor=True)
        similarity = util.cos_sim(golden_emb, interp_emb).item()

        # Save score to dataframe
        df.at[idx, "InterpretationScore"] = similarity

# Save the updated responses file
df.to_excel(chosen_output_path, index=False)
print("Done! Saved similarity scores.")


Done! Saved similarity scores.


Code in case some sentences do not get scored (There were a couple sentences that have not been recorded correctly for LLMs)

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')
golden_sentence= "A humming insect"
interpretation = ["refers to deceptive or false talk or behavior, often causing distraction or confusion.", "refers to something deceptive, fake, or misleading, often causing distraction or frustration.", " refers to something deceptive, false, or misleading, often causing distraction or confusion."]
# Embed golden sentence once
golden_emb = model.encode(golden_sentence, convert_to_tensor=True)
for inter in interpretation:
  interp_emb = model.encode(inter, convert_to_tensor=True)
  similarity = util.cos_sim(golden_emb, interp_emb).item()
  print(similarity)

0.09745556116104126
0.08139339089393616
0.06485657393932343


Pairwise scores for interpretation between the LLM responses for the same context

In [ ]:
# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Load the Excel file (already with AccuracyScore)
df = pd.read_excel("final_llm_responses_with_scores.xlsx")

# Add new column
df["AvgCohesion"] = None

# Go through the DataFrame in blocks of 5 since there are 5 responses per Context-Word combination
for i in range(0, len(df), 5):
    block = df.iloc[i:i+5]

    # Skip incomplete groups
    if len(block) < 5:
        continue

    interpretations = block["Interpretation"].tolist()

    # Encode all 5 interpretations
    embeddings = model.encode(interpretations, convert_to_tensor=True)

    # Generate all 10 unique pairwise combinations (each sentence is matched once with every other sentence)
    pairs = list(itertools.combinations(range(5), 2))
    scores = [util.cos_sim(embeddings[i], embeddings[j]).item() for i, j in pairs]

    avg_score = sum(scores) / len(scores)

    # Assign the same score to all 5 rows for easier data managing
    df.loc[i:i+4, "AvgCohesion"] = avg_score

# Save final file
df.to_excel("final_llm_responses_with_scores_and_cohesion.xlsx", index=False)
print("All AvgCohesion scores computed and saved.")

